##### Script that takes the object masks as input, converts them to features vectors using a pre-trained Res2Net model, and then cluster them using K-Means. The clusters are then saved in a new folder

In [1]:
import torch

import sys
sys.path.append("lib")
from Res2Net.res2net import Res2Net, res2net50

model = res2net50(pretrained=True)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for inference')

Using cuda for inference


In [2]:
from torchvision import transforms
from PIL import Image

# Load list of folders in the folder
# Folder is in format MOVE_neighbourhood_full_masks
folder_path = 'res/dataset/centrum_west_full/masked_extracted/' 

In [3]:
def get_array(img, model, device):
    # Apply transformations
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225]),
    ])

    model.eval()
    model.to(device)

    img = transform(img).unsqueeze(0).to(device)
    output = model(img)

    return output

In [4]:
import os

# Create a dictionary which links each left and right image with their panorama id, the future array it will be converted to,
# and the label they will get after the clustering. This is a way to not lose the information of the original image
data = {}

# The folder contains a list of folders, each one containing the masks for back, front, left and right images
# We are interested in just the left and right ones
# Ignore the .DS_Store file if it exists
imgs = ['/left.png', '/right.png']

panoramas = os.listdir(folder_path)

test_panoramas = panoramas[:30]

In [5]:
for panorama_folder in test_panoramas:
    # Create a sub-dictionary for the pano ID
    data[panorama_folder] = {}

    if panorama_folder != '.DS_Store':
        for lr in ['left', 'right']:
            # Create a sub-dictionary for the left/right image
            data[panorama_folder][lr] = {'array': None, 'label': None}
            # Load list of files in the folder
            # File is in format neighbourhood/panorama_folder/image
            file_path = folder_path + panorama_folder + f'/{lr}.png'

            # Get the feature vector
            img = Image.open(file_path)
            array = get_array(img, model, device)

            # Add the feature vector to the dictionary
            # ToDo: distinguish multiple objects in the images
            data[panorama_folder][lr]['array'] = array

In [6]:
# Save data as json file, making sure to serialize the numpy arrays
import json
import numpy as np

with open('res/dataset/centrum_west_full/masked_extracted/data.json', 'w') as fp:
    json.dump(data, fp, default=lambda o: o.tolist())
    


In [12]:
# Take a random array from the data and print its shape
import random

random_pano = random.choice(list(data.keys()))
random_lr = random.choice(list(data[random_pano].keys()))
print(data[random_pano][random_lr]['array'])

[[-0.789955735206604, -0.008090628311038017, -0.08159809559583664, -0.30156582593917847, 0.15623138844966888, -0.4778293967247009, -0.43764495849609375, -0.8523749113082886, -0.7960759401321411, 0.4071865677833557, 0.6298608183860779, -0.6983688473701477, 0.39087432622909546, -0.262079119682312, 1.0405038595199585, -0.47129496932029724, -0.7785969972610474, 0.7000814080238342, -0.1160641759634018, -0.9445639252662659, -0.46862611174583435, -0.08714693039655685, -0.24692095816135406, 0.20641455054283142, -0.22553864121437073, -0.9834144711494446, -0.4466533660888672, -0.9979915022850037, 0.33518916368484497, 0.3323114216327667, -0.8450190424919128, -0.5648680925369263, -1.158013939857483, 0.54220050573349, 1.2035659551620483, -0.2932915985584259, 0.19666869938373566, -0.14480654895305634, 0.3102022409439087, 0.023250890895724297, -0.7191408276557922, 1.411402940750122, -0.1108783483505249, -0.44602251052856445, 0.29208821058273315, 0.7119518518447876, -0.7471349835395813, 0.117468722164

In [34]:
# Clustering step
import sklearn
from sklearn.cluster import KMeans

# Load the data.json file where the feature vectors are stored
with open('res/dataset/centrum_west_full/masked_extracted/data.json') as json_file:
    data = json.load(json_file)

# Create a list of all the feature vectors
feature_vectors = []
for panorama in data:
    for lr in ['left', 'right']:
        feature_vectors.append(data[panorama][lr]['array'])

# Make the feature_vectors list a list of numpy arrays with dim 2
feature_vectors = np.array(feature_vectors)
feature_vectors = feature_vectors.reshape(feature_vectors.shape[0], -1)


# Apply KMeans clustering
clusters = 6
kmeans = KMeans(n_clusters=clusters, random_state=0).fit(feature_vectors)
labels = kmeans.predict(feature_vectors)

/home/andrealombardo/miniconda3/envs/sidewalk/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [35]:
# Add the labels to the data dictionary
for panorama in data:
    for lr in ['left', 'right']:
        data[panorama][lr]['label'] = labels[0]
        labels = labels[1:]

# Save data as json file, making sure to serialize the numpy arrays
with open('res/dataset/centrum_west_full/masked_extracted/data.json', 'w') as fp:
    json.dump(data, fp, default=lambda o: o.tolist())

In [36]:
# Make a folder in res/dataset/centrum_west_full/masked_extracted/ for each label
# Copy the images in the corresponding label folder
import shutil

for label in range(clusters):
    label_folder_path = f'res/dataset/centrum_west_full/masked_extracted/label_{label}/'
    if not os.path.exists(label_folder_path):
        os.makedirs(label_folder_path)

In [37]:
nmbr = 0
for panorama in data:
    for lr in ['left', 'right']:
        label = data[panorama][lr]['label']
        print('Panorama: ', panorama, 'LR: ', lr, 'Label: ', label)
        folder_path = f'res/dataset/centrum_west_full/masked_extracted/label_{label}/'
        file_path = folder_path + f'{lr}_{nmbr}.png'
        print('File path: ', file_path)
        shutil.copyfile(f'res/dataset/centrum_west_full/masked_extracted/{panorama}/{lr}.png', file_path)
        nmbr += 1

Panorama:  TMX7316010203-002538_pano_0000_003920 LR:  left Label:  1
File path:  res/dataset/centrum_west_full/masked_extracted/label_1/left_0.png
Panorama:  TMX7316010203-002538_pano_0000_003920 LR:  right Label:  2
File path:  res/dataset/centrum_west_full/masked_extracted/label_2/right_1.png
Panorama:  TMX7316010203-002050_pano_0001_001277 LR:  left Label:  1
File path:  res/dataset/centrum_west_full/masked_extracted/label_1/left_2.png
Panorama:  TMX7316010203-002050_pano_0001_001277 LR:  right Label:  0
File path:  res/dataset/centrum_west_full/masked_extracted/label_0/right_3.png
Panorama:  TMX7316010203-002620_pano_0002_000328 LR:  left Label:  3
File path:  res/dataset/centrum_west_full/masked_extracted/label_3/left_4.png
Panorama:  TMX7316010203-002620_pano_0002_000328 LR:  right Label:  3
File path:  res/dataset/centrum_west_full/masked_extracted/label_3/right_5.png
Panorama:  TMX7316010203-002548_pano_0000_000963 LR:  left Label:  4
File path:  res/dataset/centrum_west_full/m